# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "../output_data/cities.csv"

cities_df = pd.read_csv(cities_csv)
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
location_df0 = pd.DataFrame({"Lat": cities_df["Lat"],
           "Lng": cities_df["Lng"]})
location_df = location_df0.astype('float')
print(location_df.dtypes)

location_df['Coordinates'] = list(zip(location_df.Lat, location_df.Lng))
location_df['Humidity'] = cities_df["Humidity"]
location_df



Lat    float64
Lng    float64
dtype: object


,Lat,Lng,Coordinates,Humidity
0,68.05,39.51,"(68.05, 39.51)",72
1,-20.41,57.70,"(-20.41, 57.7)",74
2,77.48,-69.36,"(77.48, -69.36)",73
3,40.71,112.04,"(40.71, 112.04)",39
4,-33.93,18.42,"(-33.93, 18.42)",76
...,...,...,...,...
543,33.97,36.66,"(33.97, 36.66)",32
544,24.01,-104.61,"(24.01, -104.61)",4
545,49.65,0.71,"(49.65, 0.71)",62
546,51.07,132.56,"(51.07, 132.56)",93


In [5]:
# Create GMap
figure_layout = {
    'width':'400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '25px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout= figure_layout)

heatmap = gmaps.heatmap_layer(location_df["Coordinates"],weights = location_df['Humidity'],max_intensity = 3)
fig.add_layer(heatmap)
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='25px', w…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_df0 = cities_df.loc[cities_df['Max Temp']>65.00]
weather_df1 = weather_df0.loc[weather_df0['Humidity']>70]
weather_df = weather_df1.dropna()
weather_df
cities_coord1 = pd.DataFrame({"Lat": weather_df["Lat"],
                             "Lng": weather_df["Lng"]})
cities_coord = cities_coord1.astype('float')
print(cities_coord)
weather_df['Coordinates'] = list(zip(cities_coord.Lat, cities_coord.Lng))
weather_df

       Lat     Lng
1   -20.41   57.70
5    -9.80 -139.03
12  -21.21 -159.78
18  -20.34   57.76
20   20.86   92.31
..     ...     ...
522  61.99    5.14
524   2.04   45.34
536  -0.92  100.36
538  43.10    5.88
539  -0.90  119.87

[127 rows x 2 columns]


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Coordinates
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,"(-20.41, 57.7)"
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49,"(-9.8, -139.03)"
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24,"(-21.21, -159.78)"
18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41,"(-20.34, 57.76)"
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,"(20.86, 92.31)"
...,...,...,...,...,...,...,...,...,...,...,...
522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70,"(61.99, 5.14)"
524,524,mogadishu,79,SO,1558378801,79,2.04,45.34,80.88,9.78,"(2.04, 45.34)"
536,536,padang,100,ID,1558378896,82,-0.92,100.36,81.24,7.65,"(-0.92, 100.36)"
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,"(43.1, 5.88)"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = weather_df
hotel_df["Hotel Name"] = ''
hotel_df.reset_index()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Coordinates,Hotel Name
0,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,"(-20.41, 57.7)",
1,5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49,"(-9.8, -139.03)",
2,12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24,"(-21.21, -159.78)",
3,18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41,"(-20.34, 57.76)",
4,20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,"(20.86, 92.31)",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70,"(61.99, 5.14)",
123,524,524,mogadishu,79,SO,1558378801,79,2.04,45.34,80.88,9.78,"(2.04, 45.34)",
124,536,536,padang,100,ID,1558378896,82,-0.92,100.36,81.24,7.65,"(-0.92, 100.36)",
125,538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,"(43.1, 5.88)",


In [8]:
params = {'radius': 5000,
         'types': 'hotel',
         'key': g_key
         }

for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f"{lat},{lng}"
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url,params = params, verify = False).json()

    try:
        hotel_df.loc[index,"Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/App

/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Missing field/result... skipping.


/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/App

/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/App

/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/App

/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/App

/Applications/anaconda3/envs/PythonData/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Coordinates,Hotel Name
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,"(-20.41, 57.7)",Mahebourg
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49,"(-9.8, -139.03)",Atuona
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24,"(-21.21, -159.78)",Avarua District
18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41,"(-20.34, 57.76)",Vieux Grand Port
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,"(20.86, 92.31)",Teknaf
...,...,...,...,...,...,...,...,...,...,...,...,...
522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70,"(61.99, 5.14)",Raudeberg
524,524,mogadishu,79,SO,1558378801,79,2.04,45.34,80.88,9.78,"(2.04, 45.34)",Mogadishu
536,536,padang,100,ID,1558378896,82,-0.92,100.36,81.24,7.65,"(-0.92, 100.36)",Padang
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,"(43.1, 5.88)",Toulon


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
info_box = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


hotel_info = [info_box.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat","Lng"]]

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='25px', w…